In [1]:
!nvidia-smi

Fri Dec 24 08:07:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install keras_self_attention

In [3]:
import pickle
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras_self_attention import SeqSelfAttention
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import backend as K
import pandas as pd

In [4]:
features = 1280
# 2048 for ResNet50
# 1024 for DenseNet121
# 1280 for MobileNetV2
output_file = '/content/drive/MyDrive/Projects/WinterCEERI/Results_MobileNetV2.p'


with open ('/content/drive/MyDrive/Projects/WinterCEERI/saveX_MobileNetV2.p', 'rb') as fp:
  X = pickle.load(fp)
    
with open ('/content/drive/MyDrive/Projects/WinterCEERI/saveY_MobileNetV2.p', 'rb') as fp:
  Y = pickle.load(fp)

In [5]:
def pad_into_array(list, total_length):
  T = []
    
  for i in range(len(list)):
    pad = total_length - X[i].shape[0]
    if pad <= 0:
      T.append(list[i][:total_length, :])
    else:
      T.append(np.pad(list[i], [(0, pad), (0, 0)]))
  
  return np.array(T)

In [6]:
def print_plots(history, neurons, depth, model_type):

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'valid'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/Projects/WinterCEERI/Type{}_Depth{}_Neurons{}_Acc.png'.format(model_type, depth, neurons))
  plt.clf()

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'valid'], loc='upper left')
  plt.savefig('/content/drive/MyDrive/Projects/WinterCEERI/Type{}_Depth{}_Neurons{}_Loss.png'.format(model_type, depth, neurons))
  plt.clf()

In [7]:
max_length = 30

X = pad_into_array(X, max_length)

print(type(X))
print(X.shape)
print(type(Y))
print(Y.shape)

<class 'numpy.ndarray'>
(1600, 30, 1280)
<class 'numpy.ndarray'>
(1600, 11)


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1280, 30, 1280)
(1280, 11)
(320, 30, 1280)
(320, 11)


In [9]:
total_neurons = [32, 64, 128, 256]
output_classes = 11

results = pd.DataFrame(columns = ['Type', 'Depth', 'Neurons', 'Accuracy', 'Loss'])
print(results)

Empty DataFrame
Columns: [Type, Depth, Neurons, Accuracy, Loss]
Index: []


In [10]:
for neurons in total_neurons:
    
  print('Doing Type 1, Depth 1, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(neurons))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '1(LSTM)', 'Depth' : 1, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 1, 1)
    
  K.clear_session()
  
  print('Doing Type 1, Depth 2, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(neurons, return_sequences=True))
  model.add(layers.LSTM(neurons))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '1(LSTM)', 'Depth' : 2, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 2, 1)
    
  K.clear_session()
  
  print('Doing Type 1, Depth 3, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(neurons, return_sequences=True))
  model.add(layers.LSTM(neurons, return_sequences=True))
  model.add(layers.LSTM(neurons))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '1(LSTM)', 'Depth' : 3, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 3, 1)
    
  K.clear_session()

Doing Type 1, Depth 1, Neuron 32
Doing Type 1, Depth 2, Neuron 32
Doing Type 1, Depth 3, Neuron 32
Doing Type 1, Depth 1, Neuron 64
Doing Type 1, Depth 2, Neuron 64
Doing Type 1, Depth 3, Neuron 64
Doing Type 1, Depth 1, Neuron 128
Doing Type 1, Depth 2, Neuron 128
Doing Type 1, Depth 3, Neuron 128
Doing Type 1, Depth 1, Neuron 256
Doing Type 1, Depth 2, Neuron 256
Doing Type 1, Depth 3, Neuron 256


<Figure size 432x288 with 0 Axes>

In [11]:
for neurons in total_neurons:
    
  print('Doing Type 2, Depth 1, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons)))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '2(BiLSTM)', 'Depth' : 1, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 1, 2)
    
  K.clear_session()
    
  print('Doing Type 2, Depth 2, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons)))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '2(BiLSTM)', 'Depth' : 2, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 2, 2)
    
  K.clear_session()
  
  print('Doing Type 2, Depth 3, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons)))
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '2(BiLSTM)', 'Depth' : 3, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 3, 2)
    
  K.clear_session()

Doing Type 2, Depth 1, Neuron 32
Doing Type 2, Depth 2, Neuron 32
Doing Type 2, Depth 3, Neuron 32
Doing Type 2, Depth 1, Neuron 64
Doing Type 2, Depth 2, Neuron 64
Doing Type 2, Depth 3, Neuron 64
Doing Type 2, Depth 1, Neuron 128
Doing Type 2, Depth 2, Neuron 128
Doing Type 2, Depth 3, Neuron 128
Doing Type 2, Depth 1, Neuron 256
Doing Type 2, Depth 2, Neuron 256
Doing Type 2, Depth 3, Neuron 256


<Figure size 432x288 with 0 Axes>

In [12]:
for neurons in total_neurons:
    
  print('Doing Type 3, Depth 1, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(layers.Flatten())
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '3(AttBiLSTM)', 'Depth' : 1, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 1, 3)
    
  K.clear_session()
    
  print('Doing Type 3, Depth 2, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(layers.Flatten())
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '3(AttBiLSTM)', 'Depth' : 2, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 2, 3)
    
  K.clear_session()
  
  print('Doing Type 3, Depth 3, Neuron {}'.format(neurons))
  model = keras.Sequential()
  model.add(keras.Input(shape=(max_length, features)))
  model.add(layers.BatchNormalization())
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(layers.Bidirectional(layers.LSTM(neurons, return_sequences=True)))
  model.add(SeqSelfAttention(attention_activation='sigmoid'))
  model.add(layers.Flatten())
  model.add(layers.Dense(output_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  history = model.fit(X_train, Y_train, batch_size=30, epochs=20, validation_split=0.25, verbose=0)
  
  test_scores = model.evaluate(X_test, Y_test, verbose=0)
  results = results.append({'Type' : '3(AttBiLSTM)', 'Depth' : 3, 'Neurons' : neurons, 'Accuracy' : test_scores[1], 'Loss' : test_scores[0]}, ignore_index = True)
  
  print_plots(history, neurons, 3, 3)
    
  K.clear_session()

Doing Type 3, Depth 1, Neuron 32
Doing Type 3, Depth 2, Neuron 32
Doing Type 3, Depth 3, Neuron 32
Doing Type 3, Depth 1, Neuron 64
Doing Type 3, Depth 2, Neuron 64
Doing Type 3, Depth 3, Neuron 64
Doing Type 3, Depth 1, Neuron 128
Doing Type 3, Depth 2, Neuron 128
Doing Type 3, Depth 3, Neuron 128
Doing Type 3, Depth 1, Neuron 256
Doing Type 3, Depth 2, Neuron 256
Doing Type 3, Depth 3, Neuron 256


<Figure size 432x288 with 0 Axes>

In [13]:
print(results)

            Type Depth Neurons  Accuracy      Loss
0        1(LSTM)     1      32  0.950000  0.172860
1        1(LSTM)     2      32  0.928125  0.217620
2        1(LSTM)     3      32  0.950000  0.195812
3        1(LSTM)     1      64  0.956250  0.127031
4        1(LSTM)     2      64  0.931250  0.230898
5        1(LSTM)     3      64  0.950000  0.221171
6        1(LSTM)     1     128  0.956250  0.130814
7        1(LSTM)     2     128  0.950000  0.173807
8        1(LSTM)     3     128  0.959375  0.187292
9        1(LSTM)     1     256  0.959375  0.109810
10       1(LSTM)     2     256  0.968750  0.125133
11       1(LSTM)     3     256  0.950000  0.186872
12     2(BiLSTM)     1      32  0.968750  0.106863
13     2(BiLSTM)     2      32  0.959375  0.117798
14     2(BiLSTM)     3      32  0.931250  0.250760
15     2(BiLSTM)     1      64  0.959375  0.115284
16     2(BiLSTM)     2      64  0.978125  0.077231
17     2(BiLSTM)     3      64  0.934375  0.254286
18     2(BiLSTM)     1     128 

In [14]:
print(results.sort_values(by=['Accuracy'], ascending=False))

            Type Depth Neurons  Accuracy      Loss
18     2(BiLSTM)     1     128  0.978125  0.074097
16     2(BiLSTM)     2      64  0.978125  0.077231
27  3(AttBiLSTM)     1      64  0.978125  0.109450
30  3(AttBiLSTM)     1     128  0.971875  0.139901
26  3(AttBiLSTM)     3      32  0.971875  0.088888
25  3(AttBiLSTM)     2      32  0.971875  0.078439
24  3(AttBiLSTM)     1      32  0.971875  0.069527
22     2(BiLSTM)     2     256  0.971875  0.075721
28  3(AttBiLSTM)     2      64  0.968750  0.082110
10       1(LSTM)     2     256  0.968750  0.125133
12     2(BiLSTM)     1      32  0.968750  0.106863
21     2(BiLSTM)     1     256  0.968750  0.088439
34  3(AttBiLSTM)     2     256  0.965625  0.321109
23     2(BiLSTM)     3     256  0.965625  0.141517
35  3(AttBiLSTM)     3     256  0.962500  0.301745
15     2(BiLSTM)     1      64  0.959375  0.115284
31  3(AttBiLSTM)     2     128  0.959375  0.162493
19     2(BiLSTM)     2     128  0.959375  0.132277
13     2(BiLSTM)     2      32 

In [15]:
with open(output_file, 'wb') as fp:
    pickle.dump(results, fp)